# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [1]:
# #langchain library
# !pip install langchain==0.1.0
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2

# !pip install bitsandbytes==0.41.2

# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

In [2]:
import os
import torch

os.environ["TOKENIZERS_PARALLELISM"] = "True"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [3]:
from langchain import PromptTemplate

prompt_template = """
    I'm your friendly AIT chatbot named AIT-GPT, here to assist new students who are still unfamiliar with our campus.
    You can ask any questions you may have about AIT's history, policies, facilities, services or any related topics.
    I'll be sure to answer you to the best of my ability!!
    Context: {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly AIT chatbot named AIT-GPT, here to assist new students who are still unfamiliar with our campus.\n    You can ask any questions you may have about AIT's history, policies, facilities, services or any related topics.\n    I'll be sure to answer you to the best of my ability!!\n    Context: {context}\n    Question: {question}\n    Answer:")

In [4]:
PROMPT.format(
    context = "AIT's beautiful main campus located just north of Bangkok, Thailand, AIT operates as a self-contained international community with a cosmopolitan approach to living and learning.",
    question = "Where is AIT's main campus located"
)

"I'm your friendly AIT chatbot named AIT-GPT, here to assist new students who are still unfamiliar with our campus.\n    You can ask any questions you may have about AIT's history, policies, facilities, services or any related topics.\n    I'll be sure to answer you to the best of my ability!!\n    Context: AIT's beautiful main campus located just north of Bangkok, Thailand, AIT operates as a self-contained international community with a cosmopolitan approach to living and learning.\n    Question: Where is AIT's main campus located\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [5]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [6]:
# documents

In [7]:
len(documents)

95

In [8]:
documents[1]

Document(page_content=' \nTABLE OF CONTENTS \n \n \nI. \nIntroducing AIT \n \nII. \nStudent Bill of Rights \n \nIII. \nStudent Code of Conduct \n \nIV. \nGuidance for New Students \n \nV. \nStudent Welfare Unit and the AIT Career Center \n \nVI. \nHarassment Policy \n \nVII. \nSubstance Abuse Policy \n \nVIII. \nEnvironment Policy \n \nIX. \nStudent Organizations and Student Participation in Institute Governance \n- Student Union \n- Nationality Associations \n- Student Participation in Institute Governance \n \nX. \nCampus Facilities and Services \n- Student Accommodation \n- Visas \n- Banking \n- Dining \n- Sports and Recreation \n- Movies on Campus \n- Religious Services \n- Mails \n- AIT Reception \n- Office of Public Affairs (OPA) \n- Books and Office Supplies \n- English Language Training and Advising \n- Library \n- Internet and Other Campus Wide IT Services \n- Child Schooling \n- Medical Examination and Emergency \n- Security Service \n \nXI. \nGetting Around \n- On Campus \n-

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [10]:
doc[1]

Document(page_content='TABLE OF CONTENTS \n \n \nI. \nIntroducing AIT \n \nII. \nStudent Bill of Rights \n \nIII. \nStudent Code of Conduct \n \nIV. \nGuidance for New Students \n \nV. \nStudent Welfare Unit and the AIT Career Center \n \nVI. \nHarassment Policy \n \nVII. \nSubstance Abuse Policy \n \nVIII. \nEnvironment Policy \n \nIX. \nStudent Organizations and Student Participation in Institute Governance \n- Student Union \n- Nationality Associations \n- Student Participation in Institute Governance \n \nX. \nCampus Facilities and Services \n- Student Accommodation \n- Visas \n- Banking \n- Dining \n- Sports and Recreation \n- Movies on Campus \n- Religious Services \n- Mails \n- AIT Reception \n- Office of Public Affairs (OPA)', metadata={'source': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'page': 1, 'total_pages': 95, 'format': 'PDF 1.4', 'title': '(\\(NDIJobG

In [11]:
len(doc)

371

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [12]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/home/neoshocker/.local/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


/home/neoshocker/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/neoshocker/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer


/home/neoshocker/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [13]:
#locate vectorstore
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [14]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [15]:
#calling vector from local
vector_path = './vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   

In [16]:
#ready to use
retriever = vectordb.as_retriever()

In [17]:
retriever.get_relevant_documents("How much is the rental fee for dormitory")

[Document(page_content='full month payment. \n          \n2.         Self-funded students are required to pay a one-month advance deposit. For new \n \n  \n    students, the first invoice includes rental for two months. \n          \n3.         Special and exchange students are required to pay the rent in full for the whole  \n \n     duration of their stay on campus. The full payment will be reflected in the first invoice \n \n     upon registration. \n          \n4.      Rent does not include charges for utilities (electricity, water, unlimited        internet \n \n     access, etc.). \n  \n         \n5. Payment made after 30 days from the date of an invoice, will generate', metadata={'source': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'page': 55, 'total_pages': 95, 'format': 'PDF 1.4', 'title': '(\\(NDIJobGUID=fb11135f-67cd-4d80-beec-458427a81384\\)Microsoft Word 

In [18]:
retriever.get_relevant_documents("What are the accommodations available")

[Document(page_content='X. \nCAMPUS FACILITIES AND SERVICES \n \n \nStudent Accommodation on Campus \n  \nThe allocation of student accommodation is the responsibility of the \nAccommodation Unit (ext. 5093). Any issue or concern involving student  \nwelfare that is related to accommodation should be brought to the attention of \nthe Director, Office of Student Affairs (OSA), on action required. \n  \nPolicy and Procedures on Student Accommodation: Eligibility and \nAllocation \n  \nDefinition and Scope \nA student accommodation is a unit under the following dorm \ncategories Category 1:    \nDorms A, B, C, D, F, G, H \nCategory 2:    \nDorms E, J, K \nCategory 3:    \nDorms L, M, N, P3, Q, R, S \nCategory 4:', metadata={'source': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'page': 48, 'total_pages': 95, 'format': 'PDF 1.4', 'title': '(\\(NDIJobGUID=fb11135f-67cd-4d80-

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [19]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [20]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [21]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [22]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [23]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [24]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [25]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0
# %cd ..

In [26]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = './models/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [27]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [28]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [29]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [30]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/home/neoshocker/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [31]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly AIT chatbot named AIT-GPT, here to assist new students who are still unfamiliar with our campus.\n    You can ask any questions you may have about AIT's history, policies, facilities, services or any related topics.\n    I'll be sure to answer you to the best of my ability!!\n    Context: {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7f710d4f7b50>)), document_variable_name='context')

In [32]:
query = "Where is AIT's main campus located?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly AIT chatbot named AIT-GPT, here to assist new students who are still unfamiliar with our campus.
    You can ask any questions you may have about AIT's history, policies, facilities, services or any related topics.
    I'll be sure to answer you to the best of my ability!!
    Context: AIT’s main campus offers fast and easy access to Bangkok, a city at the crossroads of 
East, Southeast and South Asia. AIT has also established a key learning center in 
Vietnam. 
 
With friends all over the globe, a strong history of academic excellence, and an 
enduring reputation for responding to emerging regional and global challenges such as 
climate change and sustainability, AIT is advancing new understanding and applying 
relevant technological solutions across Asia through its knowledge hub in Thailand. 
 
AIT Quick Facts 
Students: 1,400+ from 40+ countries 
Faculty: 134 

{'input_documents': [Document(page_content='AIT’s main campus offers fast and easy access to Bangkok, a city at the crossroads of \nEast, Southeast and South Asia. AIT has also established a key learning center in \nVietnam. \n \nWith friends all over the globe, a strong history of academic excellence, and an \nenduring reputation for responding to emerging regional and global challenges such as \nclimate change and sustainability, AIT is advancing new understanding and applying \nrelevant technological solutions across Asia through its knowledge hub in Thailand. \n \nAIT Quick Facts \nStudents: 1,400+ from 40+ countries \nFaculty: 134 internationally recruited Faculty from 20+ countries', metadata={'source': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'page': 2, 'total_pages': 95, 'format': 'PDF 1.4', 'title': '(\\(NDIJobGUID=fb11135f-67cd-4d80-beec-458427a81384\\)Mic

In [33]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="I'm your friendly AIT chatbot named AIT-GPT, here to assist new students who are still unfamiliar with our campus.\n    You can ask any questions you may have about AIT's history, policies, facilities, services or any related topics.\n    I'll be sure to answer you to the best of my ability!!\n    Context: {context}\n    Question: {question}\n    Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7f710d4f7b50>)), document_variable_name='context'), question_generator=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation a

## 5. Chatbot

In [34]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly AIT chatbot named AIT-GPT, here to assist new students who are still unfamiliar with our campus.
    You can ask any questions you may have about AIT's history, policies, facilities, services or any related topics.
    I'll be sure to answer you to the best of my ability!!
    Context: involved in the claimed harassment, abuse or discrimination, and acceptable to 
both sides, (probably, though not exclusively, someone from the list in Section 
C-2.3 above). The agreed individual(s) will meet privately and/or together with 
each of the sides involved, attempting to clarify their perceptions, and to reach 
a mutually- acceptable understanding for their future interactions. Outcomes of

(https://www.ait.ac.th). For content updates, please send an email to Public Affairs. 
  
Contact: Public Affairs office is loca

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': "<pad> I  am  an  AI  chatbot  named  AIT-GPT,  I  am  here  to  assist  new  students  who  are  still  unfamiliar  with  AIT's  campus.  I  am  here  to  answer  any  questions  you  may  have  about  AIT's  history,  policies,  facilities,  services  or  any  related  topics.  I'll  be  sure  to  answer  you  to  the  best  of  my  ability!!\n",
 'source_documents': [Document(page_content='involved in the claimed harassment, abuse or discrimination, and acceptable to \nboth sides, (probably, though not exclusively, someone from the list in Section \nC-2.3 above). The agreed individual(s) will meet privately and/or together with \neach of the sides involved, attempting to clarify their perceptions, and to reach \na mutually- acceptable understanding for their future interactions. Outcomes of', metadata={'source': './docs/Student-Handbook_August-2023-Semester_FINAL_as-of-8-Aug-2023.pdf', 'file_path': './docs/Stude

In [35]:
prompt_question = "Where is AIT's main campus located?"
answer = chain({"question": prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad> I  am  an  AI  chatbot  named  AIT-GPT,  I  am  here  to  assist  new  students  who  are  still  unfamiliar  with  AIT's  campus.  I  am  here  to  answer  any  questions  you  may  have  about  AIT's  history,  policies,  facilities,  services  or  any  related  topics.  I'll  be  sure  to  answer  you  to  the  best  of  my  ability!!\n")]
Follow Up Input: Where is AIT's main campus located?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I'm your friendly AIT chatbot named AIT-GPT, here to assist new students who are still unfamiliar with ou

{'question': "Where is AIT's main campus located?",
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content="<pad> I  am  an  AI  chatbot  named  AIT-GPT,  I  am  here  to  assist  new  students  who  are  still  unfamiliar  with  AIT's  campus.  I  am  here  to  answer  any  questions  you  may  have  about  AIT's  history,  policies,  facilities,  services  or  any  related  topics.  I'll  be  sure  to  answer  you  to  the  best  of  my  ability!!\n")],
 'answer': '<pad>  just  north  of  Bangkok,  Thailand\n',
 'source_documents': [Document(page_content='AIT’s main campus offers fast and easy access to Bangkok, a city at the crossroads of \nEast, Southeast and South Asia. AIT has also established a key learning center in \nVietnam. \n \nWith friends all over the globe, a strong history of academic excellence, and an \nenduring reputation for responding to emerging regional and global challenges such as \nclimate change and sustainability, AIT is advan

In [36]:
prompt_question = "Does it have any other campus?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?'), AIMessage(content="<pad> I  am  an  AI  chatbot  named  AIT-GPT,  I  am  here  to  assist  new  students  who  are  still  unfamiliar  with  AIT's  campus.  I  am  here  to  answer  any  questions  you  may  have  about  AIT's  history,  policies,  facilities,  services  or  any  related  topics.  I'll  be  sure  to  answer  you  to  the  best  of  my  ability!!\n"), HumanMessage(content="Where is AIT's main campus located?"), AIMessage(content='<pad>  just  north  of  Bangkok,  Thailand\n')]
Follow Up Input: Does it have any other campus?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt 

{'question': 'Does it have any other campus?',
 'chat_history': [HumanMessage(content='Who are you by the way?'),
  AIMessage(content="<pad> I  am  an  AI  chatbot  named  AIT-GPT,  I  am  here  to  assist  new  students  who  are  still  unfamiliar  with  AIT's  campus.  I  am  here  to  answer  any  questions  you  may  have  about  AIT's  history,  policies,  facilities,  services  or  any  related  topics.  I'll  be  sure  to  answer  you  to  the  best  of  my  ability!!\n"),
  HumanMessage(content="Where is AIT's main campus located?"),
  AIMessage(content='<pad>  just  north  of  Bangkok,  Thailand\n')],
 'answer': '<pad> Yes,  AIT  has  established  a  key  learning  center  in  Vietnam.\n',
 'source_documents': [Document(page_content='AIT’s main campus offers fast and easy access to Bangkok, a city at the crossroads of \nEast, Southeast and South Asia. AIT has also established a key learning center in \nVietnam. \n \nWith friends all over the globe, a strong history of academi